<a href="https://colab.research.google.com/github/flari-gold/IU/blob/main/IU_RedditDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('Hello World!!!!')

In [ ]:
import requests

from google.colab import userdata

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(userdata.get('CLIENT_ID'), userdata.get('CLIENT_SECRET'))

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': userdata.get('USERNAME'),
        'password': userdata.get('PASSWORD')}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
#requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

In [ ]:
subreddit = 'finanzen'
limit = 5
timeframe = 'week' #hour, day, week, month, year, all
listing = 'top' # controversial, best, hot, new, random, rising, top

url = f"https://oauth.reddit.com/r/{subreddit}/{listing}?limit={limit}&t={timeframe}"
res = requests.get(url, headers=headers)

import json
print(json.dumps(res.json(), indent=2))

In [8]:
subreddit = 'finanzen'
limit = 5
timeframe = 'week' #hour, day, week, month, year, all
listing = 'top' # controversial, best, hot, new, random, rising, top
id = '1gmifua'

#http://www.reddit.com/r/" + sub + "/comments/" + id + ".json?",
url = f"https://oauth.reddit.com/r/{subreddit}/comments/{id}.json?"
res = requests.get(url, headers=headers)

import json
print(json.dumps(res.json(), indent=2))

[
  {
    "kind": "Listing",
    "data": {
      "after": null,
      "dist": 1,
      "modhash": null,
      "geo_filter": "",
      "children": [
        {
          "kind": "t3",
          "data": {
            "approved_at_utc": null,
            "subreddit": "Finanzen",
            "selftext": "",
            "user_reports": [],
            "saved": false,
            "mod_reason_title": null,
            "gilded": 0,
            "clicked": false,
            "title": "Ein Gl\u00fccksgriff im Leben",
            "link_flair_richtext": [
              {
                "e": "text",
                "t": "Investieren - Aktien"
              }
            ],
            "subreddit_name_prefixed": "r/Finanzen",
            "hidden": false,
            "pwls": 6,
            "link_flair_css_class": "Investieren - Aktien",
            "downs": 0,
            "top_awarded_type": null,
            "hide_score": false,
            "name": "t3_1gmifua",
            "quarantine": false,
     